In [2]:
# 导入需要的包
import pandas as pd
import numpy as np
from sklearn.feature_selection import chi2,SelectKBest,f_classif
import warnings  
warnings.filterwarnings("ignore")  

In [3]:
Target_data = pd.read_csv('Traindata_clean.csv',sep=',',encoding='utf-8')

In [4]:
Target_data.head(2)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,1,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0


In [55]:
Y = Target_data['SeriousDlqin2yrs'].values
X = Target_data.iloc[:,1:]

In [56]:
Target_data['SeriousDlqin2yrs'].value_counts()

1    136125
0      9712
Name: SeriousDlqin2yrs, dtype: int64

In [57]:
X.head(2)

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0


In [16]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score

## 基于SelectKBest建立模型，筛选变量个数

In [9]:
def Select_Feature(data_X,data_Y):
    score_estimator ={}
    for i in range(1,data_X.shape[1]+1):
        # 建立变量选择模型
        SK = SelectKBest(k=i)
        # 获取指定数量的特征
        X_new =SK.fit_transform(data_X,data_Y)
        # 建立分类模型进行评估
        clf =LogisticRegressionCV(cv=5,penalty='l2',scoring=None,random_state=2018, solver='lbfgs').fit(X_new,data_Y)
        score =clf.score(X_new,data_Y)
        score_estimator[i] =score
    # 模型得分进行排序取最大值的特征个数
    best_FeatureNum =sorted(score_estimator.items(),key=lambda x:x[1],reverse= True)[0][0]
    SK = SelectKBest(k=best_FeatureNum)
    # 获取指定最优数量的特征
    X_Best =SK.fit_transform(data_X,data_Y)
    print("原始数据集的变量个数{0}".format(data_X.shape[1]))
    print("SelectKBest筛选后数据集的变量个数{0}".format(X_Best.shape[1]))
    return X_Best

In [10]:
X_new =Select_Feature(X,Y)

原始数据集的变量个数10
SelectKBest筛选后数据集的变量个数8


In [11]:
clf =LogisticRegressionCV(cv=5,penalty='l2',scoring=None,random_state=2018, solver='lbfgs').fit(X_new,Y)
score =clf.score(X_new,Y)
print("SelectKbest模型的得分是:",score)

SelectKbest模型的得分是: 0.9366004511886559


## 基于SelectFromModel建立模型，筛选变量个数

In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [58]:
from sklearn.model_selection import train_test_split
def Select_FromModel(data_X,data_Y,split_num):
    accuracy_score ={}
    model = RandomForestClassifier(n_estimators=100,random_state=2018,n_jobs=6)
    for i in range(1,13):
        # 模型得分进行排序取最大值的特征个数
        sfm = SelectFromModel(model,threshold="{0}*mean".format(float(i*0.1)))
        # 获取指定最优数量的特征
        X_New =sfm.fit_transform(data_X,data_Y)
        X_train, X_test, Y_trian, Y_test = train_test_split(X_New,data_Y,test_size=0.2,stratify=data_Y)
        model.fit(X_train, Y_trian)
        predicted = model.predict(X_test)
        print("i ={0}时准确率为：{1}".format(i,sum(predicted == Y_test)/len(Y_test)))
        accuracy_score[i] = sum(predicted == Y_test)/len(Y_test)
    # 模型得分进行排序取最大值的特征个数
    best_Num =sorted(accuracy_score.items(),key=lambda x:x[1],reverse= True)[0][0]
    # 模型得分进行排序取最大值的特征个数
    sfm = SelectFromModel(model,threshold="{0}*mean".format(float(best_Num*0.1)))
    # 获取指定最优数量的特征
    X_Best =sfm.fit_transform(data_X,data_Y)
    model.fit(X_Best,data_Y)
    print(sfm.threshold)
    print("原始数据集的变量个数{0}".format(data_X.shape[1]))
    print("SelectKBest筛选后数据集的变量个数{0} and threshold is {1}*mean".format(X_Best.shape[1],best_Num))
    print("SelectKbest模型的得分是:",accuracy_score[best_Num])
    return X_Best

In [59]:
X_New = Select_FromModel(data_X = X,data_Y = Y,split_num=4)

i =1时准确率为：0.9375685682940208
i =2时准确率为：0.9373971475589687
i =3时准确率为：0.936848601206802
i =4时准确率为：0.9376714207350522
i =5时准确率为：0.9373971475589687
i =6时准确率为：0.9380828304991772
i =7时准确率为：0.9373628634119583
i =8时准确率为：0.9372257268239166
i =9时准确率为：0.9380142622051564
i =10时准确率为：0.9345858475041141
i =11时准确率为：0.9347572682391662
i =12时准确率为：0.9348258365331871
None 0.6000000000000001*mean
原始数据集的变量个数10
SelectKBest筛选后数据集的变量个数6 and threshold is 6*mean
SelectKbest模型的得分是: 0.9380828304991772


In [23]:
importance =[0.18957967, 0.11849264, 0.04741798, 0.17546375, 0.17272516, 0.08761128,0.09011804 ,0.03274487, 0.04472129, 0.04112531]

In [13]:
clf =RandomForestClassifier(n_estimators=100,random_state=2018,n_jobs=6)
# 模型得分进行排序取最大值的特征个数
sfm = SelectFromModel(clf,threshold= "mean")
X_new1 =sfm.fit_transform(X,Y)

In [14]:
clf.fit(X,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)

In [15]:
clf.feature_importances_

array([0.18957967, 0.11849264, 0.04741798, 0.17546375, 0.17272516,
       0.08761128, 0.09011804, 0.03274487, 0.04472129, 0.04112531])

In [133]:
clf.score(X,Y)

0.9997805769454939

In [17]:
X_new =sfm.fit_transform(X,Y)

In [18]:
sfm.get_support()

array([ True,  True, False,  True,  True, False, False, False, False,
       False])

In [131]:
sfm.threshold_

0.1

In [34]:
from sklearn.datasets import make_classification
import pandas as pd 
from collections import Counter

In [45]:
X,y = make_classification(n_samples = 1000,n_features = 2,n_informative=2,n_redundant=0,n_repeated=0,
                         n_classes = 2,n_clusters_per_class=1,weights =[0.1,0.9],class_sep=0.8,random_state=0)

In [46]:
X_train, X_test, Y_trian, Y_test = train_test_split(X,y,test_size=0.2,stratify=y)

In [47]:
Counter(Y_trian)

Counter({1: 718, 0: 82})

In [48]:
Counter(y)

Counter({1: 897, 0: 103})